In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=64745794e33c130a41aeccdb69a8d149cecc3dcfbb071ad7a4c2509319b74038
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 4.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F7 '''
def F7(X):
    f = bn.F7()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.129 ,random_state=0 , l1_ratio=0.8989, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 26.03 , C = 819.14 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_7_2000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_7_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=894062, Mon Apr 13 13:55:38 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.843645786828523e+03 1.0e+00 2.56e-01  3e-01  3e-01 0:00.0
    2     30 9.843607900325373e+03 1.0e+00 2.50e-01  2e-01  3e-01 0:00.0
    3     45 9.843602121350315e+03 1.1e+00 2.48e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9844.07739857] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9843.62855731] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -1.963428856846806e+03 1.4e+00 5.42e-02  5e-02  5e-02 0:01.9
  200   3000 -3.415265888213626e+03 1.5e+00 3.77e-03  3e-03  4e-03 0:03.8
  300   4500 -3.421427994960104e+03 1.5e+00 2.16e-04  2e-04  2e-04 0:05.7
  400   6000 -3.421446330734145e+03 1.6e+00 1.62e-05  1e-05  1e-05 0:07.6
  500   7500 -3.421446422434428e+03 1.6e+00 1.26e-06  1e-06  1e-06 0:09.6
  600   9000 -3.421446422910454e+03 1.6e+00 8.73e-08  7e-08  8e-08 0:11.5
  700  10500 -3.421446422912408e+03 1.7e+00 5.01e-09  4e-09  4e-09 0:13.4
  800  12000 -3.421446422912415e+03 1.8e+00 1.44e-09  1e-09  1e-09 0:15.3
  876  13140 -3.421446422912411e+03 2.0e+00 1.37e-09  1e-09  1e-09 0:16.7
termination on tolfun=1e-11 (Mon Apr 13 13:55:58 2020)
final/bestever f-value = -3.421446e+03 -3.421446e+03
incumbent solution: [-0.04640023 -0.03560729 -0.03945414 -0.04082889  0.05080723 -0.09815586
  0.14197103 -0.00179389 ...]
std deviations: [1.03594703e-09 1.06044112e-09 1.01820327e-09 1.20132833e-09
 1.01751280e-09 1.02249796

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9685.78555929] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=9685.785559, sigma=2.56e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=9685.785559, sigma=2.56e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -3.730642073729061e+09 1.6e+00 1.32e+02  1e+02  1e+02 0:01.8
  109   1635 -1.430461043620654e+10 1.6e+00 2.72e+02  3e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 13:56:00 2020)
final/bestever f-value = -1.332256e+10 -1.430461e+10
incumbent solution: [ -293.36980443  1617.37005654   328.47499621  -411.89719097
  -499.01005853   823.82376655 -1190.07739036   911.35017233 ...]
std deviations: [262.16486098 262.81841713 261.01817732 280.58976637 263.09960264
 264.1575005  268.02661012 266.10219793 ...]
Run : 1
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=914504, Mon Apr 13 13:56:00 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.841869778283226e+03 1.0e+00 2.55e-01  3e-01  3e-01 0:00.0
    2     30 9.843127919915392e+03 1.0e+00 2.47e-01  2e-01  2e-01 0:00.0
    3     45 9.842328460504670e+03 1.1e+00 2.41e-01  2e-01  2e-01 0:00.1


...

  100   1500 -3.933753855282643e+09 1.4e+00 1.25e+02  1e+02  1e+02 0:01.8
  110   1650 -1.677876365335456e+10 1.4e+00 2.72e+02  3e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 14:05:54 2020)
final/bestever f-value = -1.565543e+10 -1.677876e+10
incumbent solution: [-539.90343218 1665.32375011  -21.76576677   40.87437866  -65.81450687
 -874.84267313 -369.34958615 -191.73749352 ...]
std deviations: [269.49488342 264.96019961 259.09272806 259.04534329 266.98550153
 263.44703363 259.93191184 261.881387   ...]
Run : 29
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=861748, Mon Apr 13 14:05:54 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.843761039435303e+03 1.0e+00 2.54e-01  3e-01  3e-01 0:00.0
    2     30 9.843572022467490e+03 1.0e+00 2.46e-01  2e-01  2e-01 0:00.0
    3     45 9.843236527649886e+03 1.0e+00 2.43e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9844.93344798] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9844.31173353] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -2.640860425661253e+03 1.5e+00 3.98e-02  4e-02  4e-02 0:01.9
  200   3000 -3.418734353733716e+03 1.5e+00 2.60e-03  2e-03  3e-03 0:03.8
  300   4500 -3.421440420069501e+03 1.5e+00 1.18e-04  1e-04  1e-04 0:05.7
  400   6000 -3.421446390090154e+03 1.5e+00 9.05e-06  7e-06  8e-06 0:07.6
  500   7500 -3.421446422787392e+03 1.6e+00 5.59e-07  4e-07  5e-07 0:09.6
  600   9000 -3.421446422912019e+03 1.7e+00 3.33e-08  2e-08  3e-08 0:11.5
  700  10500 -3.421446422912411e+03 1.8e+00 2.99e-09  2e-09  3e-09 0:13.4
  787  11805 -3.421446422912411e+03 2.0e+00 1.46e-09  1e-09  1e-09 0:15.1
termination on tolfun=1e-11 (Mon Apr 13 14:06:11 2020)
final/bestever f-value = -3.421446e+03 -3.421446e+03
incumbent solution: [-0.04640022 -0.0356073  -0.03945414 -0.0408289   0.05080724 -0.09815586
  0.14197104 -0.0017939  ...]
std deviations: [1.09557352e-09 1.09289747e-09 1.15667395e-09 1.15524126e-09
 1.12695129e-09 1.16342438e-09 1.13851107e-09 1.13043838e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9685.78555929] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=9685.785559, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=9685.785559, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -6.929227086502951e+09 1.5e+00 1.67e+02  2e+02  2e+02 0:01.8
  105   1575 -1.561872435284315e+10 1.5e+00 2.70e+02  3e+02  3e+02 0:01.9
termination on tolfacupx=1000.0 (Mon Apr 13 14:06:14 2020)
final/bestever f-value = -1.499769e+10 -1.561872e+10
incumbent solution: [-405.18999605  770.50757494  470.31504259 -680.24024725 -398.26843468
  225.2346903  -743.34931178  556.57432114 ...]
std deviations: [260.01443155 259.79012414 254.8923281  267.3348522  262.52262169
 263.5868249  260.59893084 261.32734287 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F7_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F7(X_Values.iloc[i,:50])
    SVM_Fun [i] = F7(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F7(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([5113.25036261]), count=array([29])) 102.82588784873127
SVM
ModeResult(mode=array([4073.26800906]), count=array([1])) 410.8354338763059
ELN
ModeResult(mode=array([1.58160155e+08]), count=array([1])) 68700995.62926015
